# Load configuration

In [2]:
import os
import yaml


with open("../conf/service.dev.yaml", 'r') as f:
    configs = yaml.safe_load(f)
os.environ['OPENAI_API_KEY'] = configs['openai']['api_key']

---

# Chain = Prompt | Model | Parser

## Grocery
## 1. Prompt
```
langchain.prompts.ChatPromptTemplate.from_messages()
langchain.prompts.ChatPromptTemplate.from_template()
```

## 2. Model
```
langchain_openai.ChatOpenAI()
langchain_core.utils.function_calling.convert_to_openai_function()
```

## 3. Parser
```
langchain.schema.output_parser.StrOutputParser()
langchain.agents.output_parsers.OpenAIFunctionsAgentOutputParser()
from langchain.output_parsers.openai_functions.JsonOutputFunctionsParser()
from langchain.output_parsers.openai_functions.JsonKeyOutputFunctionsParser()
```

---

## 1. Basic chain
1. Prompt
    - `from_template`
2. Model
    - `ChatOpenAI`
3. Parser
    - `StrOutputParser`

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser


prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
parser = StrOutputParser()
chain = prompt | model | parser

chain.invoke({'topic': "bears"})

"Why don't bears wear shoes?\n\nBecause they already have bear feet!"

## 2. Function calling
1. Prompt
    - `from_messages`
2. Model
    - `ChatOpenAI`
    - `functions`
3. Parser
    - `OpenAIFunctionsAgentOutputParser`

### 2.1 Define input class(`pydantic`)

In [36]:
from pydantic import BaseModel, Field


class WikipediaInput(BaseModel):
    query: str = Field(description="Query to search")
    n_contents: int = Field(description="Number of contents to return")

### 2.2 Define python function(`tool`)

In [37]:
from langchain.agents import tool
import wikipedia
from wikipedia.exceptions import PageError, DisambiguationError


@tool(args_schema=WikipediaInput)
def search_wikipedia(query: str, n_contents: int) -> str:
    """Run Wikipedia search and get page summaries."""
    titles = wikipedia.search(query)
    summaries = []

    for title in titles[:n_contents]:
        try:
            page = wikipedia.page(title=title, auto_suggest=False)
            summary = page.summary
            summaries.append(f"[{title}]\n"
                             f"{summary}")
        except (PageError, DisambiguationError):
            pass
    
    if not summaries:
        return "No good Wikipedia Search Result was found"
    
    return (50*"=").join(summaries)

In [41]:
search_wikipedia({'query': "langchain", 'n_contents': 1})

"[LangChain]LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis."

### 2.3 Convert to OpenAI function(schema)

In [6]:
from langchain_core.utils.function_calling import convert_to_openai_function

tools = [search_wikipedia]
# tools = [WikipediaInput]  # also possible
functions = [convert_to_openai_function(tool) for tool in tools]
functions

[{'name': 'search_wikipedia',
  'description': 'search_wikipedia(query: str, n_contents: int) -> str - Run Wikipedia search and get page summaries.',
  'parameters': {'type': 'object',
   'properties': {'query': {'description': 'Query to search',
     'type': 'string'},
    'n_contents': {'description': 'Number of contents to return',
     'type': 'integer'}},
   'required': ['query', 'n_contents']}}]

### 2.4 Build model

#### 2.4.1 Bind function

In [ ]:
from langchain_openai import ChatOpenAI


model = ChatOpenAI()
model = model.bind(
    functions=functions
)

#### 2.4.2 Bind function + Forcing
- 다른 방법들에 비해 `prompt_tokens` 개수가 가장 많으니 주의

In [ ]:
from langchain_openai import ChatOpenAI


model = ChatOpenAI()
model = model.bind(
    functions=functions,
    function_call={'name': "search_wikipedia"}
)

#### 2.4.3 Bind function + Forcing not to use

In [ ]:
from langchain_openai import ChatOpenAI


model = ChatOpenAI()
model = model.bind(
    functions=functions,
    function_call='none'
)

### 2.5 Build chain

In [6]:
from langchain.prompts import ChatPromptTemplate
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser


prompt = ChatPromptTemplate.from_messages([
    ('system', "You are helpful but sassy assistant"),
    ('user', "{input}")
])
parser = OpenAIFunctionsAgentOutputParser()
chain = prompt | model | parser

print(chain.invoke({'input': "Hi!"}))
print(chain.invoke({'input': "what is langchain?"}))

return_values={'output': 'Hello! How can I assist you today?'} log='Hello! How can I assist you today?'
tool='search_wikipedia' tool_input={'query': 'langchain', 'n_contents': 1} log="\nInvoking: `search_wikipedia` with `{'query': 'langchain', 'n_contents': 1}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "langchain",\n  "n_contents": 1\n}', 'name': 'search_wikipedia'}})]


### Example) Tagging & Extraction

#### - Tagging
텍스트로부터 구조화된 데이터를 추출

In [9]:
from pydantic import BaseModel, Field
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser


class Tagging(BaseModel):
    """Tag the piece of the text with particular info."""
    sentiment: str = Field(description="sentiment of text, should be `pos`, `neg`, or `neutral`") 
    language: str = Field(description="language of text (should be ISO 639-1 code)")


prompt = ChatPromptTemplate.from_messages([
    ('system', "Think carefully, and then tag the text as instructed"),
    ('user', "{input}")
])
model = ChatOpenAI().bind(
    functions=[convert_to_openai_function(Tagging)],
    function_call={'name': "Tagging"}
)
parser = JsonOutputFunctionsParser()
chain = prompt | model | parser

chain.batch((
    {'input': "I love langchain"},
    {'input': "non mi piace questo cibo"},
    {'input': "이 식당은 밥을 적정량 준다"}
))

[{'sentiment': 'pos', 'language': 'en'},
 {'sentiment': 'neg', 'language': 'it'},
 {'sentiment': 'neutral', 'language': 'ko'}]

#### - Extraction
Tagging과 비슷하지만 **다수**의 정보를 추출한다.

In [38]:
from pydantic import BaseModel, Field
from typing import Optional, List
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser


class Person(BaseModel):
    """Information about a person."""
    name: str = Field(description="person's name")
    age: Optional[int] = Field(description="person's age")

class Information(BaseModel):
    """Information to extract."""
    people: List[Person] = Field(description="List of info about people")


prompt = ChatPromptTemplate.from_messages([
    ('system', "Extract the relevant information, if not explicitly provided do not guess and stay empty. Extract partial info"),
    ('user', "{input}")
])
model = ChatOpenAI().bind(
    functions=[convert_to_openai_function(Information)],
    function_call={'name': "Information"}
)
parser = JsonOutputFunctionsParser(key_name='people')
chain = prompt | model | parser

chain.batch((
    {'input': "Joe is 30, his mom is Martha who is 20 years older than Joe"},
    {'input': "민철이는 30살이고, 그보다 20살 많은 어머니의 이름은 희진입니다"}
))

[{'people': [{'name': 'Joe', 'age': 30}, {'name': 'Martha', 'age': 0}]},
 {'people': [{'name': '민철', 'age': 30}, {'name': '희진', 'age': 50}]}]

- 값에 `20+30` 과 같이 수식이 들어간 경우, json parser가 인식하지 못한다.
    - 이를 해결할 방법을 찾을 예정

In [39]:
args = '{\n "people": [\n    {\n      "name": "Joe",\n      "age": 30\n    },\n    {\n      "name": "Martha",\n      "age": 30 + 20\n    }\n  ]\n}'

In [36]:
eval(args)

{'people': [{'name': 'Joe', 'age': 30}, {'name': 'Martha', 'age': 50}]}

#### - Document extraction

In [29]:
from langchain.document_loaders import WebBaseLoader
    

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
documents = loader.load()
doc = documents[0]
doc.page_content[:1000]

"\n\n\n\n\n\nLLM Powered Autonomous Agents | Lil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPosts\n\n\n\n\nArchive\n\n\n\n\nSearch\n\n\n\n\nTags\n\n\n\n\nFAQ\n\n\n\n\nemojisearch.app\n\n\n\n\n\n\n\n\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\n \n\n\nTable of Contents\n\n\n\nAgent System Overview\n\nComponent One: Planning\n\nTask Decomposition\n\nSelf-Reflection\n\n\nComponent Two: Memory\n\nTypes of Memory\n\nMaximum Inner Product Search (MIPS)\n\n\nComponent Three: Tool Use\n\nCase Studies\n\nScientific Discovery Agent\n\nGenerative Agents Simulation\n\nProof-of-Concept Examples\n\n\nChallenges\n\nCitation\n\nReferences\n\n\n\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspi

In [30]:
from pydantic import BaseModel, Field
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser


class Overview(BaseModel):
    """Overview of a section of text."""
    summary: str = Field(description="Provide a concise summary of the content.")
    language: str = Field(description="Provide the language that the content is written.")
    keywords: str = Field(description="Provide keywords.")


prompt = ChatPromptTemplate.from_messages([
    ('system', "You are helpful summary expert for document. Extract the information from the given text."),
    ('human', "{input}")
])
model = ChatOpenAI().bind(
    functions=[convert_to_openai_function(Overview)],
    function_call={'name': "Overview"}
)
chain = prompt | model | JsonOutputFunctionsParser()

chain.invoke({'input': doc.page_content[:10000]})

{'summary': 'This document provides an overview of LLM-powered autonomous agents, including their components, such as planning, memory, and tool use. It discusses various techniques used in each component, such as task decomposition, self-reflection, and external API calls. The document also mentions proof-of-concept examples and challenges associated with building LLM-powered agents.',
 'language': 'English',
 'keywords': 'LLM, autonomous agents, planning, memory, tool use, task decomposition, self-reflection, external API calls, proof-of-concept examples, challenges'}

In [32]:
from pydantic import BaseModel, Field
from typing import Optional, List
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser


class Paper(BaseModel):
    """Information about papers mentioned."""
    title: str = Field(description="Title of the paper without authors")
    author: Optional[str] = Field(description="Author of the paper")

class Information(BaseModel):
    """Information to extract."""
    papers: List[Paper] = Field(description="List of papers to extract from the text")

template = """A article will be passed to you. Extract from it all papers that are mentioned by this article.
Do not extract the name of the article itself. If no papers are mentioned that's fine - you don't need to extract any! Just return an empty list.
Do not make up or guess ANY extra information. Only extract what exactly is in the text."""
prompt = ChatPromptTemplate.from_messages([
    # ('system', "You are helpful paper extractor for the document. Think carefully then extract the information from the given text. Think step by step."),
    ('system', template),
    ('human', "{input}")
])
model = ChatOpenAI(temperature=0).bind(
    functions=[convert_to_openai_function(Information)],
    function_call={'name': "Information"}
)
chain = prompt | model | JsonKeyOutputFunctionsParser(key_name='papers')

chain.invoke({'input': doc.page_content[:10000]})

[{'author': 'Wei et al. 2022'},
 {'author': 'Yao et al. 2023'},
 {'author': 'Liu et al. 2023'},
 {'author': 'Shinn & Labash 2023'},
 {'author': 'Laskin et al. 2023'},
 {'author': 'Duan et al. 2017'}]

- `title`은 required 인데도 잘 안 되는 경우가 있다.

In [33]:
convert_to_openai_function(Information)

{'name': 'Information',
 'description': 'Information to extract.',
 'parameters': {'type': 'object',
  'properties': {'papers': {'description': 'List of papers to extract from the text',
    'type': 'array',
    'items': {'description': 'Information about papers mentioned.',
     'type': 'object',
     'properties': {'author': {'description': 'Author of the paper',
       'type': 'string'}},
     'required': ['title']}}},
  'required': ['papers']}}

- 긴 문서는 잘라서 넣고 나중에 합친다.
    - `langchain.text_splitter.RecursiveCharacterTextSplitter`

In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.runnable import RunnableLambda


def flatten(matrix):
    flat_list = []
    for row in matrix:
        flat_list += row
    return flat_list


text_splitter = RecursiveCharacterTextSplitter(chunk_overlap=0)
splits = text_splitter.split_text(doc.page_content)
prep = RunnableLambda(lambda x: [{'input': doc} for doc in text_splitter.split_text(x)])
split_chain = prep | chain.map() | flatten

split_chain.invoke(doc.page_content)

[{'author': 'Lilian Weng'},
 {'author': 'Wei et al. 2022'},
 {'author': 'Yao et al. 2023'},
 {'author': 'Liu et al. 2023'},
 {'author': 'Shinn & Labash 2023'},
 {'author': 'Shinn & Labash, 2023'},
 {'author': 'Liu et al. 2023'},
 {'author': 'Laskin et al. 2023'},
 {'author': 'Laskin et al. 2023'},
 {'author': 'Duan et al. 2017'},
 {'author': 'Miller 1956'},
 {'author': 'LSH (Locality-Sensitive Hashing)'},
 {'author': 'ANNOY (Approximate Nearest Neighbors Oh Yeah)'},
 {'author': 'HNSW (Hierarchical Navigable Small World)'},
 {'author': 'FAISS (Facebook AI Similarity Search)'},
 {'author': 'ScaNN (Scalable Nearest Neighbors)'},
 {'author': 'Karpas et al. 2022'},
 {'author': 'Parisi et al. 2022'},
 {'author': 'Schick et al. 2023'},
 {'author': 'Shen et al. 2023'},
 {'author': 'Li et al. 2023'},
 {'author': 'Bran et al. 2023'},
 {'author': 'Boiko et al. (2023)'},
 {'author': 'Park, et al. (2023)'},
 {'author': 'Park et al.'},
 {'author': 'John Smith'},
 {'author': 'Jane Doe'},
 {'author': 

# 3. Fallback handling

1. `json.loads` 가 정확한  json format을 입력받지 못해 에러가 발생

In [7]:
from langchain.llms import OpenAI
import json


main_model = OpenAI(
    temperature=0,
    max_tokens=1000,
    model='gpt-3.5-turbo-instruct'
)
main_chain = main_model | json.loads

challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"
main_chain.invoke(challenge)

/home/vscode/.local/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


JSONDecodeError: Extra data: line 9 column 1 (char 125)

2. 최신 LLM 사용방식은 정확한 json format을 줄 수 있음
    1. `langchain.llms.OpenAI`의 output은 `str`이고, \
    `langchain_openai.ChatOpenAI`의 output은 `AIMessage` 객채
    2. Output의 format을 `str`로 맞춰주기 위해 `StrOutputParser` 추가

In [8]:
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
import json


fallback_model = ChatOpenAI(temperature=0)
fallback_chain = fallback_model | StrOutputParser() | json.loads
fallback_chain.invoke(challenge)

{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': 'In the stillness of night, whispers of the wind'},
 'poem2': {'title': 'Silent Tears',
  'author': 'Jacob Anderson',
  'first_line': 'Silent tears fall, unseen and unheard'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Lee',
  'first_line': 'Beneath the moonlight, dancing shadows sway'}}

3. 정상작동하는 2번 방법을 fallback method로 지정

In [9]:
final_chain = main_chain.with_fallbacks([fallback_chain])
final_chain.invoke(challenge)

{'poem1': {'title': 'Whispers of the Wind',
  'author': 'Emily Rivers',
  'first_line': 'Softly it comes, the whisper of the wind'},
 'poem2': {'title': 'Silent Serenade',
  'author': 'Jacob Moore',
  'first_line': 'In the stillness of night, a silent serenade'},
 'poem3': {'title': 'Dancing Shadows',
  'author': 'Sophia Anderson',
  'first_line': 'Shadows dance upon the moonlit floor'}}

# 4. Batch processing
가능한 batch 처리되는 프로세스들은 병렬적으로 실행된다. (default `n_procs=5`)

In [10]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser


prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
parser = StrOutputParser()
chain = prompt | model | parser

chain.batch([
    {'topic': "bears"},
    {'topic': "frogs"}
])

["Why don't bears wear shoes?\n\nBecause they have bear feet!",
 'Why don\'t frogs make good loan officers?\n\nBecause they always end up in "debt"!']

# 5. Streaming processing
적은 지연시간은 더 나은 사용자 경험을 만들어낼 수 있다.

In [11]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser


prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
parser = StrOutputParser()
chain = prompt | model | parser

for t in chain.stream({'topic': "bears"}):
    print(t)


Why
 don
't
 bears
 wear
 shoes
?


Because
 they
 already
 have
 bear
 feet
!



# 6. Asynchronous methods
용도에 따라 비동기적 처리도 가능 (`ainvoke`, `abatch`, `astream`)

In [12]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser


prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
parser = StrOutputParser()
chain = prompt | model | parser

response = await chain.ainvoke({'topic': "bears"})
response

"Why don't bears wear shoes?\n\nBecause they have bear feet!"

# 7. Routing
사용되는 tool에 따라 다른 방식으로 처리할 수 있다.

In [131]:
from pydantic import BaseModel, Field
from langchain.agents import tool
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser


class PositiveInput(BaseModel):
    persons: List[str] = Field(description="Name list of persons having positive action")

@tool(args_schema=PositiveInput)
def positive(persons: List[str]) -> str:
    """Handle persons having only positive action."""
    return f"[Positive] {persons}"

class NegativeInput(BaseModel):
    persons: List[str] = Field(description="Name list of persons having negative action")

@tool(args_schema=NegativeInput)
def negative(persons: List[str]) -> str:
    """Handle persons having only negative action."""
    return f"[Negative] {persons}"

class NeutralInput(BaseModel):
    persons: List[str] = Field(description="Name list of persons having negative action")

@tool(args_schema=NeutralInput)
def neutral(persons: List[str]) -> str:
    """Handle persons having only neutral action."""
    return f"[Neutral] {persons}"


prompt = ChatPromptTemplate.from_messages([
    ('system', "You are expert of discrimating persons with emotional action. Positive or negative or neutral."),
    ('human', "{input}")
])
model = ChatOpenAI(temperature=0).bind(
    functions=[convert_to_openai_function(fn) for fn in [positive, negative, neutral]]
)
parser = OpenAIFunctionsAgentOutputParser()

chain = prompt | model | parser

In [132]:
result1 = chain.invoke({'input': "Jimmy like to eat vegatables and Sansu wants to listen piano."})

# result = chain.batch([
#     {'input': "Jimmy like to eat vegatables and Sansu wants to listen piano."},
#     {'input': "Jimmy does not eat vegatables and Sansu hates to listen piano."},
#     {'input': "Jimmy looks at Sansu."}
# ])
type(result1), result1.tool, result1.tool_input

(langchain_core.agents.AgentActionMessageLog,
 'positive',
 {'persons': ['Jimmy', 'Sansu']})

In [133]:
result2 = chain.invoke({'input': "Hi, how are you?"})
type(result2), result2.return_values

(langchain_core.agents.AgentFinish,
 {'output': "Hello! I'm an AI assistant, so I don't have feelings, but I'm here to help you. How can I assist you today?"})

In [134]:
from langchain.schema.agent import AgentFinish

def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        return eval(result.tool).run(result.tool_input)

In [135]:
chain = prompt | model | parser | route

In [136]:
chain.batch([
    {'input': "Jimmy like to eat vegatables and Sansu wants to listen piano."},
    {'input': "Hi, how are you?"}
])

["[Positive] ['Jimmy', 'Sansu']",
 "Hello! I'm an AI assistant, so I don't have feelings, but I'm here to help you. How can I assist you today?"]

- 한 문장 안에서 여러 개의 tool이 사용되는 경우는 처리하지 못하는 모양

In [140]:
chain.invoke({'input': """
              Jimmy does not eat vegatables but Hammy likes to eat vegatables.
              Minsu likes to play piano but Sansu hates to listen it.
              """})

"[Positive] ['Hammy']"

# 8. ChatBot
Multi-turn 대화를 위해 `placeholder`, `memory`를 사용

- 감정 분석 chain

In [151]:
from pydantic import BaseModel, Field
from langchain.agents import tool
import wikipedia
from wikipedia.exceptions import PageError, DisambiguationError
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser


class WikipediaInput(BaseModel):
    query: str = Field(description="Query to search")
    n_contents: int = Field(description="Number of contents to return")

@tool(args_schema=WikipediaInput)
def search_wikipedia(query: str, n_contents: int) -> str:
    """Run Wikipedia search and get page summaries."""
    titles = wikipedia.search(query)
    summaries = []

    for title in titles[:n_contents]:
        try:
            page = wikipedia.page(title=title, auto_suggest=False)
            summary = page.summary
            summaries.append(f"[{title}]\n"
                             f"{summary}")
        except (PageError, DisambiguationError):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return (50*"=").join(summaries)


prompt = ChatPromptTemplate.from_messages([
    ('system', "You are helpful but sassy assistant"),
    ('user', "{input}")
])
model = ChatOpenAI(temperature=0).bind(
    functions=[convert_to_openai_function(search_wikipedia)]
)
parser = OpenAIFunctionsAgentOutputParser()
chain = prompt | model | parser

In [149]:
chain.invoke({'input': "Hi!"})

AgentFinish(return_values={'output': 'Hello! How can I assist you today?'}, log='Hello! How can I assist you today?')

In [150]:
chain.invoke({'input': "what is langchain?"})

AgentActionMessageLog(tool='search_wikipedia', tool_input={'query': 'Langchain', 'n_contents': 1}, log="\nInvoking: `search_wikipedia` with `{'query': 'Langchain', 'n_contents': 1}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "Langchain",\n  "n_contents": 1\n}', 'name': 'search_wikipedia'}})])

In [ ]:
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents import AgentExecutor
from langchain.prompts import MessagePlaceholder
from langchain.schema.agent import AgentFinish
from langchain.schema.runnable import RunnablePassthrough


def run_agent(user_input):
    